In [2]:
import torch
import torch.nn as nn
import torch.utils.data
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import sys
sys.path.append("../")
import x3x3.frozen as frozen

data = np.load("../ising81x81_temp2.269.npy")

# Batch size, channels, height, width

# train on 3 x 3
class SupervisedConvNet(nn.Module):
    def __init__(self, filter_size, square_size, hidden_size, num_hidden_layers, center,
                first_activation = "tanh", activation_func = "sigmoid",
                out_channels = 1):
        """
        Arguments:
        filter_size ~ size of the convolution kernel (3 x 3)
        square size ~ how many strides of convolution in the input
        """
        super(SupervisedConvNet, self).__init__()
        self.filter_size = filter_size
        self.square_size = square_size
        self.hidden_size = hidden_size
        self.out_channels = out_channels
        if first_activation == "tanh":
            self.first_activation = torch.tanh
        elif first_activation == "relu":
            self.first_activation = torch.nn.LeakyReLU(0.1)
        if activation_func == "sigmoid":
            self.activation_func = torch.sigmoid
        elif activation_func == "relu":
            self.activation_func = torch.nn.LeakyReLU(0.1)

        self.conv1 = nn.Conv2d(1, out_channels, filter_size, padding=0, stride = filter_size)
        # parameters
        param = {}
        if center == "keep":
            param['conv2d.weight'] = torch.tensor([[[[1.0, 1.0, 1.0],
                      [1.0, 1.0, 1.0],
                      [1.0, 1.0, 1.0]]]])/9
            param['conv2d.bias'] = torch.tensor([0.0])
            self.conv1.bias = torch.nn.Parameter(param['conv2d.bias'], requires_grad=False)
        elif center == "omit":
            param['conv2d.weight'] = torch.tensor([[[[1.0, 1.0, 1.0],
                      [1.0, 0.0, 1.0],
                      [1.0, 1.0, 1.0]]]])/8
            # param['conv2d.bias'] = torch.tensor([0.0])
        elif center == "pre_trained":
            param['conv2d.weight'] = torch.tensor([[[[-0.0900, -0.0407, -0.0965],
          [-0.0413, -0.0011, -0.0363],
          [-0.0906, -0.0344, -0.0908]]]])
            param['conv2d.bias'] = torch.tensor([-0.0345])
            self.conv1.bias = torch.nn.Parameter(param['conv2d.bias'], requires_grad=False)
        self.conv1.weight = torch.nn.Parameter(param['conv2d.weight'], requires_grad=False)


        self.first_linear = nn.Linear(self.out_channels * square_size ** 2, hidden_size)
        hidden_layer = [nn.Linear(hidden_size, hidden_size) for _ in range(num_hidden_layers)]
        self.linear_hidden = nn.ModuleList(hidden_layer)
        self.linear_output = nn.Linear(hidden_size, 1)



    def forward(self, x):
        x = self.first_activation(self.conv1(x))#.view(-1, 1, self.out_channels * self.square_size**2)
#         x = self.activation_func(self.first_linear(x))
#         for linear in self.linear_hidden:
#             x = self.activation_func(linear(x))
#         x = torch.sigmoid(self.linear_output(x))
#         x = x.squeeze(1)

        return x


def coarse_grain_frozen_conv(correlated_data = data):
    run_mode = "frozen_convolution_pretrained_relu"
    mode = "run"
    if mode == "run":
        hidden_size = 10
        out_channels = 1
        num_hidden_layers = 1


        if run_mode == "frozen_convolution_pretrained_relu":
            """
            """
            run_num = 1
            filename = f"frozen_convolution_pretrained_relu_{run_num}.pl"
            out_channels = 1
            try:
                with open(filename, "rb") as handle:
                    results = pickle.load(handle)
            except:
                results = []
            results = []
            model = SupervisedConvNet(filter_size = 3, square_size = 27, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    center = "pre_trained", first_activation = "tanh",
                    activation_func = "relu", num_hidden_layers = num_hidden_layers)
            data = torch.tensor(correlated_data).unsqueeze(1).type('torch.FloatTensor')
            out = model(data)

    return out


In [3]:
out = coarse_grain_frozen_conv()


In [4]:
data[0, :10, :10]

array([[ 1,  1, -1,  1, -1, -1, -1, -1,  1, -1],
       [ 1,  1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 1,  1, -1, -1, -1,  1,  1,  1,  1, -1],
       [-1, -1, -1, -1, -1, -1,  1,  1,  1, -1],
       [ 1, -1, -1, -1, -1,  1,  1,  1,  1, -1],
       [-1, -1, -1, -1, -1,  1,  1,  1,  1, -1],
       [-1, -1, -1, -1, -1,  1,  1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1,  1,  1,  1,  1],
       [-1, -1,  1, -1, -1, -1, -1, -1,  1,  1],
       [ 1,  1,  1,  1, -1, -1, -1, -1,  1,  1]])

In [5]:
out[0, 0, :10, :10]

tensor([[-0.1086,  0.1249, -0.1365,  0.4520,  0.2860,  0.4502,  0.3924,  0.4520,
          0.2217,  0.4520],
        [ 0.3839,  0.2289, -0.5052,  0.4520,  0.3924,  0.1422,  0.4520,  0.3924,
          0.1401,  0.4520],
        [ 0.2964,  0.2860, -0.0318,  0.0709,  0.4520,  0.3956,  0.2328,  0.4520,
          0.4520,  0.4520],
        [-0.0382,  0.2979,  0.0400, -0.2911,  0.2209,  0.4502,  0.4520,  0.4520,
         -0.1003,  0.4520],
        [ 0.3849,  0.3937,  0.4520,  0.0418,  0.4520,  0.2180,  0.4520,  0.4520,
          0.3849,  0.4520],
        [ 0.3228,  0.1393,  0.4520,  0.2979,  0.4520,  0.2860,  0.2209,  0.2328,
          0.2325,  0.2964],
        [-0.0170,  0.4520,  0.3956,  0.4520,  0.4520,  0.4520,  0.3830, -0.1086,
          0.2096,  0.2979],
        [ 0.2979,  0.2948,  0.4520,  0.4520,  0.4520,  0.4520,  0.2860,  0.2979,
          0.4520,  0.4520],
        [ 0.4520,  0.1389,  0.4520,  0.4520,  0.2979,  0.4520,  0.4520,  0.4520,
          0.4520,  0.4520],
        [ 0.4520,  

In [6]:
out.squeeze(1).numpy().shape

(10000, 27, 27)

In [7]:
def generate_uncorrelated_data(data):
    # data expects an array of shape [10000, 27, 27]
    uncorrelated_data = []
    for _ in range(10000):
        sample = np.random.randint(0, 10000, (9, 9))
        horizontal, vertical = np.random.randint(0, 25, (2, 9, 9))
        uncorrelated = []
        for i in range(9):
            tile = []
            for j in range(9):
                tile.append(data[sample[i, j], horizontal[i, j]:(horizontal[i, j] + 3), \
                        vertical[i, j]:(vertical[i, j] + 3)])
            uncorrelated.append(np.hstack(tile))
        uncorrelated_data.append(np.vstack(uncorrelated))

    uncorrelated_data = np.array(uncorrelated_data)
    return uncorrelated_data

uncorrelated_data = generate_uncorrelated_data(out.squeeze(1).numpy())

In [8]:
uncorrelated_data[0, :10, :10]

array([[-0.5051525 , -0.44206944, -0.5051525 ,  0.2967938 ,  0.45199126,
         0.45199126,  0.45199126,  0.38387796,  0.04417124, -0.34802958],
       [-0.21956593, -0.5035121 ,  0.12533742,  0.2328493 ,  0.45199126,
         0.45199126,  0.45199126,  0.45199126,  0.2328493 , -0.04975887],
       [-0.04017835,  0.22203915,  0.22716673, -0.21632744,  0.15337996,
         0.45199126,  0.45199126,  0.45199126,  0.45199126,  0.45199126],
       [-0.5051525 , -0.5051525 , -0.5051525 , -0.5051525 , -0.5051525 ,
        -0.4521504 , -0.03019083, -0.2745702 , -0.5035121 ,  0.22203915],
       [-0.5051525 , -0.5035121 , -0.44206944, -0.29697618, -0.3580087 ,
        -0.19102426, -0.20792347, -0.5051525 , -0.5051525 ,  0.45199126],
       [-0.5051525 , -0.5051525 , -0.5051525 , -0.2845261 , -0.5051525 ,
        -0.28544486, -0.22697704, -0.13909245, -0.5051525 ,  0.45199126],
       [ 0.03997869,  0.04337278,  0.45199126,  0.15123099, -0.2069665 ,
        -0.5051525 ,  0.22089803,  0.45199126

In [12]:
out.shape

torch.Size([10000, 1, 27, 27])

In [13]:
# save files
np.save("ising27x27coarsegrained_temp2.269.npy", out.squeeze(1).numpy())
np.save("ising27x27coarsegrained_temp2.269_uncorrelated.npy", uncorrelated_data)